환경변수 가져오기

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [14]:
import os
import json
from datetime import datetime
from uuid import uuid4
from typing import Optional

from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import Runnable
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from utils import astream_graph  # 기존 스트리밍 함수

# LLM 모델
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# MCP 서버 연결
client = MultiServerMCPClient({
    "playwright": {
        "url": "http://localhost:8005/sse",
        "transport": "sse",
    }
})

# 결과 저장 디렉토리
BASE_DIR = "./results"
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
test_id = f"{timestamp}_{uuid4().hex[:6]}"
output_dir = os.path.join(BASE_DIR, test_id)
os.makedirs(output_dir, exist_ok=True)

# Pydantic 모델 + 파서
class WebTestResult(BaseModel):
    success: bool = Field(..., description="성공 여부")
    duration: Optional[float] = Field(..., description="테스트 수행 시간")
    feedback: str = Field(..., description="웹페이지 상태 피드백")

parser = PydanticOutputParser(pydantic_object=WebTestResult)

# 요약용 프롬프트 템플릿
summary_prompt = PromptTemplate(
    template="""
다음 웹 자동화 테스트 로그를 바탕으로 결과를 아래 형식의 JSON으로 요약해줘:
{format_instructions}

로그:
{raw_result}
""",
    input_variables=["raw_result"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

summary_chain: Runnable = summary_prompt | model | parser

# 🔄 로그 누적 리스트
collected_text_chunks = []

# 콜백 함수에서 로그 수집
async def collect_text(event: dict):
    content = event["content"]
    if hasattr(content, "content"):
        collected_text_chunks.append(content.content)
    else:
        collected_text_chunks.append(str(content))

# 전체 실행 함수
async def run_test():
    await client.__aenter__()
    tools = client.get_tools()
    agent = create_react_agent(model, tools)

    # message 정의는 그대로 유지
    message = {
        "scenarios": [
            {
                "title": "네이버 검색 기능 확인",
                "steps": [
                    "https://naver.com 에 접속한다.",
                    "검색창에 SSAFY를 입력하고 검색버튼을 누른다.",
                    "SSAFY와 관련된 게시물이 나오는지 확인한다."
                ]
            },
            {
                "title": "구글에서 이미지 검색 테스트",
                "steps": [
                    "https://www.google.com 에 접속한다.",
                    "'cute cats'를 검색하고 상단 탭에서 '이미지'를 클릭한다.",
                    "이미지 썸네일이 정상적으로 로딩되는지 확인한다."
                ]
            },
            {
                "title": "유튜브 영상 검색 기능 테스트",
                "steps": [
                    "https://www.youtube.com 에 접속한다.",
                    "'lofi hip hop'을 검색창에 입력하고 Enter를 누른다.",
                    "검색 결과에 썸네일과 제목이 정상적으로 표시되는지 확인한다."
                ]
            }
        ]
    }
    
    DEFAULT_RESULT_STEP = "성공 여부, 속도, 피드백을 포함해 결과를 요약한다."

    results = []

    # 시나리오들을 자연어 메시지로 변환
    for scenario in message["scenarios"]:
        # 각 시나리오 메시지 구성
        steps = scenario["steps"] + [DEFAULT_RESULT_STEP]
        steps_text = "\n".join([f"{i+1}. {step}" for i, step in enumerate(steps)])
        messages = f"### {scenario['title']}\n{steps_text}"

        # 로그 누적 리스트 초기화
        collected_text_chunks.clear()

        # LangGraph 실행 (스트리밍 로그 수집)
        await astream_graph(
            agent,
            inputs={"messages": messages},
            stream_mode="messages",
            callback=collect_text
        )

        joined_result = "\n".join(collected_text_chunks)

        # 요약 결과 생성
        parsed = await summary_chain.ainvoke({"raw_result": joined_result})

        # 저장할 형태로 추가
        results.append({
            "title": scenario["title"],
            "success": parsed.success,
            "duration": parsed.duration,
            "feedback": parsed.feedback
        })


    # 전체 결과 저장
    result_json_path = os.path.join(output_dir, "result.json")
    with open(result_json_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print("요약 결과 저장 완료:", result_json_path)

# 실행
await run_test()


요약 결과 저장 완료: ./results\20250422-122519_ff2329\result.json


그래프를 실행하여 결과를 확인합니다.